The signals are the main components in vessim for feeding simulations its data. This can be, for instance, a live signal for solar irradience data or a signal retrieving data from a historical time-series dataset on carbon intensity of the public grid. The abstract base class thereby contains a single function `at()` to be overwritten by its subclasses. It takes a datetime or a datetime-like string as well as some optional key-word arguments, and returns the data point of the signal at the given time.
```python
class Signal(ABC):
    """Abstract base class for signals."""

    @abstractmethod
    def at(self, dt: DatetimeLike, **kwargs):
        """Retrieves actual data point at given time."""
```        

To extract data from a historical dataset, the HistoricalSignal from vessim can be used. In addition to actual data, this class also supports the retrieval of forecasted data. A toy example would look like this:

In [1]:
import pandas as pd
from vessim.signal import HistoricalSignal

# Mock actual data at two different places
# The data doesn't have to be sorted and can contain NaN values
actual_data = [
   ["2020-01-01T00:00:00", 100, 800],
   ["2020-01-01T00:30:00", 110, 850],
   ["2020-01-01T01:00:00", 105, 820],
   ["2020-01-01T01:30:00", 108, 840],
]
actual = pd.DataFrame(
   actual_data, columns=["timestamp", "zone_a", "zone_b"]
)
actual.set_index(["timestamp"], inplace=True)

# Depending on the type of forecast, there can be either one or two indices
# In this case, the first column marks the time when the forecast was made
# an the second one marks the time the forecast was made for
forecast_data = [
   ["2020-01-01T00:00:00", "2020-01-01T00:30:00", 115, 850],
   ["2020-01-01T00:00:00", "2020-01-01T01:00:00", 110, 870],
   ["2020-01-01T00:00:00", "2020-01-01T01:30:00", 110, 860],
   ["2020-01-01T00:00:00", "2020-01-01T02:00:00", 120, 830],
   ["2020-01-01T01:00:00", "2020-01-01T01:30:00", 115, 840],
   ["2020-01-01T01:00:00", "2020-01-01T02:00:00", 110, 830],
]
forecast = pd.DataFrame(
   forecast_data, columns=["req_time", "forecast_time", "zone_a", "zone_b"]
)

# The order of the indices is important here
forecast.set_index(["req_time", "forecast_time"], inplace=True)

custom_signal = HistoricalSignal(actual, forecast, fill_method="ffill")


Now, it is possible to query actual and forecasted data, using this class.

In [2]:
# `at` in between timestamps returns the previous value because of the specified fill_method
custom_signal.at("2020-01-01T00:30:00", column="zone_a")

110.0

In [3]:
# The start_time is excluded in the result and the data at the last available request_time before
# the start_time  is used (in this case all forecasts made at time 2020-01-01T00:00:00 )
custom_signal.forecast("2020-01-01T00:30:00", "2020-01-01T02:00:00", column="zone_b")

{numpy.datetime64('2020-01-01T01:00:00.000000000'): 870.0,
 numpy.datetime64('2020-01-01T01:30:00.000000000'): 860.0,
 numpy.datetime64('2020-01-01T02:00:00.000000000'): 830.0}

In [4]:
# There is the possibility to specify a frequency and a resample_method
# The actual value of the specified zone at the start_time is used for linear interpolation
custom_signal.forecast("2020-01-01T00:15:00", "2020-01-01T01:40:00", column="zone_a", frequency="20min", resample_method="linear")

{numpy.datetime64('2020-01-01T00:35:00.000000000'): 114.16666666666667,
 numpy.datetime64('2020-01-01T00:55:00.000000000'): 110.83333333333333,
 numpy.datetime64('2020-01-01T01:15:00.000000000'): 110.0,
 numpy.datetime64('2020-01-01T01:35:00.000000000'): 111.66666666666667}

Vessim does also have a couple of datasets that can be directly loaded into a HistoricalSignal with the possibility of shifting and scaling. Please refer to the documentation for more information on the available datasets.

In [5]:
dataset_signal = HistoricalSignal.from_dataset("solcast2022_global", params={"scale": 2.0, "start_time":"2020-06-12T00:00:00"})
dataset_signal.at("2020-06-27T13:46:53", column="Berlin")

Required data files not present locally. Try downloading...
Successfully downloaded and unpacked data files.
Data is being shifted by -727 days +23:55:00


1.289502